In [ ]:
!pip install tensorflow
!pip install scikeras

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler # Import MinMaxScaler here
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense # Import LSTM and Dense here
from scikeras.wrappers import KerasClassifier, KerasRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from tensorflow.keras.optimizers import Adam

optimizer = Adam(learning_rate=0.001)  # Set learning rate here
model.compile(optimizer=optimizer, loss='mse')

# Select relevant columns for the analysis
data = drone_df_edited[['time', 'battery_current', 'battery_voltage', 'payload', 'altitude', 'speed', 'wind_speed', 'wind_angle']]

# Define dependent (y) and independent (X) variables
X = data[['payload', 'altitude', 'speed', 'wind_speed', 'wind_angle']]
y = data[['battery_current', 'battery_voltage']]

# Scale data using MinMaxScaler
# Prefered for LSTM
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()
X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# Create a dataset with lookback
# Should help predict battery current and voltage by remembering what happened previously
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), :-2] # Independent variables
        X.append(a)
         # Dependent variables (battery_current, battery_voltage)
        Y.append(dataset[i + look_back, -2:])
    return np.array(X), np.array(Y)

# Number of previous steps to consider
look_back = 10
X, y = create_dataset(np.concatenate([X, y], axis=1), look_back)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=100))
# Output layer with 2 neurons (battery_current, battery_voltage)
model.add(Dense(units=2))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
# model.fit(X_train, y_train, epochs=10, batch_size=32)

# y_pred_10_epochs = model.predict(X_test)
# y_pred_10_epochs = scaler_y.inverse_transform(y_pred_10_epochs)
# mse_10_epochs = np.mean(np.square(y_test - y_pred_10_epochs))
# print(f"Mean Squared Error (10 epochs): {mse_10_epochs}")


# Train the model with 25 epochs and calculate new MSE
# model.fit(X_train, y_train, epochs=25, batch_size=32) # Changed epochs to 25
# y_pred_25_epochs = model.predict(X_test)
# y_pred_25_epochs = scaler_y.inverse_transform(y_pred_25_epochs)
# mse_25_epochs = np.mean(np.square(y_test - y_pred_25_epochs))
# print(f"Mean Squared Error (25 epochs): {mse_25_epochs}")

# Train the model with 50 epochs and model units = 100, change the optimizer
model.fit(X_train, y_train, epochs=50, batch_size=32)
y_pred_50_epochs = model.predict(X_test)
y_pred_50_epochs = scaler_y.inverse_transform(y_pred_50_epochs)
mse_50_epochs = np.mean(np.square(y_test - y_pred_50_epochs))
print(f"Mean Squared Error (50 epochs) and batch size = 32: {mse_50_epochs}")